In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('prescriber-info.csv')

In [3]:
df.head(2)

,NPI,Gender,State,Credentials,Specialty,ABILIFY,ACETAMINOPHEN.CODEINE,ACYCLOVIR,ADVAIR.DISKUS,AGGRENOX,...,VERAPAMIL.ER,VESICARE,VOLTAREN,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE,Opioid.Prescriber
0,1710982582,M,TX,DDS,Dentist,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1245278100,F,AL,MD,General Surgery,0,0,0,0,0,...,0,0,0,0,0,0,0,0,35,1


In [4]:
df.dropna(inplace=True)

In [5]:
df.shape

(24237, 256)

In [6]:
from sklearn import cross_validation, svm
import numpy as np

c:\users\kimbe\appdata\local\programs\python\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
pos= df[(df["Opioid.Prescriber"]==1)]
neg= df[(df["Opioid.Prescriber"]==0)]

In [8]:
pos= pos.sample(n=9000)
neg= neg.sample(n=9000)

In [9]:
tot= pd.concat([neg, pos])

In [10]:
tot_special=tot.Specialty.unique()
tot_creds= tot.Credentials.unique()

In [11]:
def credSpecDict():
    cred_dict={}
    spec_dict={}
    for i in range(len(tot_creds)):
        cred_dict[i]= tot_creds[i]
    for i in range(len(tot_special)):
        spec_dict[i]= tot_special[i]
    return cred_dict, spec_dict

In [12]:
c, s= credSpecDict()

In [13]:
print(list(s.keys())[list(s.values()).index('Dentist')])

2


In [14]:
tot.Gender= tot.Gender.replace(['M', 'F'], [1, 0])
tot.Credentials=tot.Credentials.replace(tot_creds, [i for i in range(len(tot_creds))])
tot.Specialty=tot.Specialty.replace(tot_special, [i for i in range(len(tot_special))])

In [15]:
tot.shape

(18000, 256)

In [15]:
X = np.array(tot.drop(['State', 'NPI', 'Opioid.Prescriber'], 1))
y = np.array(tot['Opioid.Prescriber'])

In [16]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.2)

In [17]:
from sklearn.ensemble import RandomForestClassifier
clf2 = RandomForestClassifier(min_impurity_split=0.05)
clf2.fit(X_train, y_train)
print(clf2.score(X_test, y_test))

c:\users\kimbe\appdata\local\programs\python\python36\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
c:\users\kimbe\appdata\local\programs\python\python36\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
c:\users\kimbe\appdata\local\programs\python\python36\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
c:\users\kimbe\appdata\local\programs\python\python36\lib\site-packages\sklearn\tree\tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be remo

0.9238888888888889


In [18]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
print("Number of mislabeled points out of a total {} points : {}, performance {:05.2f}%"
      .format(
          X_test.shape[0],
          (y_test != y_pred).sum(),
          100*(1-(y_test != y_pred).sum()/X_test.shape[0])
))

Number of mislabeled points out of a total 3600 points : 292, performance 91.89%


In [19]:
gnb.predict_proba(X_test)[0][1]

1.0

In [20]:
def extractFeatures():
    features= list(tot.columns)
    removeFeatures= ['NPI', 'State', 'Opioid.Prescriber']
    for i in features:
        if i in removeFeatures:
            features.remove(i)
    return features

In [21]:
def returnPreds(row):
    y=gnb.predict(row)
    return int(y)

In [22]:
new=tot.drop(['State', 'NPI', 'Opioid.Prescriber'],axis= 1)

In [23]:
new.reset_index(inplace=True, drop=True)

In [24]:
new=new[extractFeatures()]

In [25]:
len(new.head(1))

1

In [26]:
np.array(new.iloc[0])

array([ 1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0, 11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
       12,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0

In [27]:
d= pd.read_csv('demo4.csv')

In [28]:
len(np.array(d.iloc[0]))

254

In [32]:
def fileEntry():
        # open csv file, read in dataframe
        # add predictions to df then save it as new file
        file =  filedialog.askopenfilename(initialdir = "/", \
        title = "Select file",filetypes = (("csv files","*.csv"),("all files","*.*")))
        valList=[]
        if file != None:
            data = pd.read_csv(file)
            data=data[extractFeatures()]
            for i, row in data.iterrows():
                val= returnPreds([np.array(data.iloc[i])])
                if val==1:
                    val= "Pill Mill"
                else:
                    val = "Not Pill Mill"
                valList.append(val)
            v=pd.Series(valList)
            data['preds']= v.values
#             data.to_csv('predictions.csv')
            print('done')

In [33]:
from tkinter import filedialog

In [34]:
fileEntry()

done


In [38]:
from tkinter import *